# microTVM Ahead-of-Time (AOT) Compilation
本教學展示了使用 TensorFlow 模型進行 microTVM 主機驅動的 AoT 編譯，並使用 C Runtime (CRT) 在 x86 CPU 上執行。與 GraphExecutor 相比，AoTExecutor 減少了運行時解析圖的開銷。此外，我們可以使用提前編譯來實現更好的記憶體管理。

**AOT Executor 的特性：**
AOT executor 產生的 C code 會包含一組函式介面，像是 tvmgen_default_run()、tvmgen_default_set_input()、tvmgen_default_get_output() 等函式。使用者可以在一個獨立的 C 程式碼中。

- 包含 model.c 或是將其編譯為物件檔後連結
- 呼叫這些函式來設定輸入、執行推論、取得輸出

同時也需要少量的 CRT runtime 支援檔案（通常是一些 platform.c/h 和 memory alloc 等簡單函式）。不過這不再是大型的 libtvm_runtime.so，而是幾個可以輕鬆整合進你專案的單檔 C 程式碼。

## 1. TVM 載入 ONNX 模型
使用 ONNX 套件載入事先預訓練好的 DNN 鳶尾花分類模型。並透過 TVM 將 ONNX 模型轉換為 Relay 模組。

In [3]:
import onnx
import tvm
from tvm import relay

# 載入 ONNX 模型
onnx_model = onnx.load("./model/rfc/deploy_model.onnx")
onnx_model = onnx.load("./model/logistic/modified_model.onnx")
# onnx_model = onnx.load("./modified_model.onnx")

# 定義輸入資訊
input_name = 'float_input'
shape_dict = {input_name: (1, 4)}

# 將 ONNX 模型轉換為 Relay 模組
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

## 1.1 修改 ONNX 模型(替換運算子)
TVM 的 C backend 對於某些運算子的支援度可能不足。遇到的 Unresolved call Op(tir.fabs) 錯誤，明確指出 C backend 無法處理絕對值運算 (fabs)。

以下是如何替換節點的範例，例如將 Abs 運算子替換成 Sqrt。

In [2]:
import onnx
from onnx import helper, numpy_helper

onnx_model = onnx.load("./model/logistic/deploy_model.onnx")
graph = onnx_model.graph

# 使用倒序迴圈，避免刪除節點影響後續索引
for i in reversed(range(len(graph.node))):
    node = graph.node[i]
    if node.op_type == "Abs":
        input_name = node.input[0]
        output_name = node.output[0]

        # 建立 Mul 節點
        mul_node = helper.make_node(
            "Mul",
            inputs=[input_name, input_name],
            outputs=[input_name + "_squared"],
            name=node.name + "_mul" if node.name else "mul_" + str(i)
        )

        # 建立 Sqrt 節點
        sqrt_node = helper.make_node(
            "Sqrt",
            inputs=[input_name + "_squared"],
            outputs=[output_name],
            name=node.name + "_sqrt" if node.name else "sqrt_" + str(i)
        )

        # 將新節點插入到原 Abs 節點的位置
        graph.node.insert(i, sqrt_node)
        graph.node.insert(i, mul_node)

        # 刪除原 Abs 節點
        graph.node.pop(i + 2)  # 注意索引變化

onnx.checker.check_model(onnx_model)
onnx.save(onnx_model, "./model/logistic/modified_model.onnx")

## 2. 使用 AOT Executor 與 C Runtime
如果希望生成一份不依賴 `libtvm_runtime.so` 的純 C 原始碼來進行推論，可以考慮使用 TVM 的 AOT (Ahead-of-Time) Executor 搭配 "crt" (C runtime)。這種組合會產生一組相對獨立的 C 程式碼，並且執行階段環境較為簡化，不需要額外動態連結到 TVM 的 runtime shared library。

在進行 TVM 編譯時，可以指定 AOT Executor 和 CRT Runtime，並啟用 `link-params`，讓模型參數直接嵌入程式碼內，免去額外的參數檔案。此外，在建立 AOT Executor 時，透過將 `interface-api` 設定為 `c` 以及將 `unpacked-api` 設定為 `True`，可以要求 TVM 生成一套清晰的 C 函式介面以及對應的標頭檔案，方便外部呼叫與整合。

需要注意的是，在 CodeGen 階段通常會啟用向量化優化，這可能導致生成的程式碼包含 GCC 或 Clang 特定的向量語法，例如 float3、float5 的向量型態初始化，這些屬於非標準 C 語法。為了解決這個問題，可以在 PassContext 中禁用向量化的設定 `tir.disable_vectorize`，這樣生成的程式碼就不會包含類似 float3、float5 的向量初始化。

In [4]:
from tvm.relay.backend import Executor, Runtime

executor = Executor("aot", {
    "interface-api": "c",    # 使用 C 接口而非 packed
    "unpacked-api": True,    # 使用 unpacked 函式簽章（參數直接用 C 函式參數方式傳遞）
    "link-params": True      # 將參數嵌入程式中
})
runtime = Runtime("crt")
# 設定目標為嵌入式設備，使用 MicroTVM
target = "c"
# target = tvm.target.target.micro("host")

with tvm.transform.PassContext(opt_level=0, config={"tir.disable_vectorize": True}):
    lib = relay.build(mod, target=target, runtime=runtime, executor=executor, params=params)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
[08:48:21] /home/jovyan/project/ONNX-MLIR/tvm/src/te/schedule/bound.cc:119: not in feed graph consumer = compute(p0_red_temp, body=[T.reduce(T.comm_reducer(lambda argmax_lhs_0, argmax_lhs_1, argmax_rhs_0, argmax_rhs_1: (T.Select(argmax_lhs_1 > argmax_rhs_1 or argmax_lhs_1 == argmax_rhs_1 and argmax_lhs_0 < argmax_rhs_0, argmax_lhs_0, argmax_rhs_0), T.Select(argmax_lhs_1 > argmax_rhs_1, argmax_lhs_1, argmax_rhs_1)), [-1, T.float32(-340282346638528859811704183484516925440.0)]), source=[k1, p0[ax0, k1]], init=[], axis=[T.iter_var(k1, T.Range(0, 3), "CommReduce", "")], condition=T.bool(True), value_index=0), T.reduce(T.comm_reducer(lambda argmax_lhs_0, argmax_lhs_1, argmax_rhs_0, argmax_rhs_1: (T.Select(argmax_lhs_1 > argmax_rhs_1 or argmax_lhs_1 == argmax_rhs_1 and argmax_lhs_0 < argmax_rhs_0, argmax_lhs_0, argmax_rhs_0), T.Select(argmax_lhs_1 > argmax_rhs

如此產生的 AOT 程式碼會對應生成更容易使用的 C 接口函式。
> 上述 link-params=True 會將參數直接打包進產生的 C 程式碼中。如此一來，你就不需要另外載入 params.bin。

## 3. 使用 MLF 格式匯出
microTVM 的相關功能，這個函式會將模型以 MLF 格式輸出。然後你解壓縮 model.tar 就能看到 .h 檔等檔案。

In [5]:
from tvm.micro import export_model_library_format
export_model_library_format(lib, "model.tar")

PosixPath('model.tar')

## 4. C 實作前置作業
### 4.1 解壓縮 MLF
將 model.tar 解壓縮後，你會看到很多的資料夾與檔案。展開後的結構大致會包含：
- codegen/host/include/tvmgen_default.h：你的模型函式介面(header檔)
- codegen/host/src/default_lib0.c、default_lib1.c：模型核心計算程式碼
- runtime/：TVM CRT (C Runtime) 所需的原始碼和標頭檔案
- parameters/default.params：模型的參數 (如果沒有 link-params=true 時會需要)

In [6]:
!rm -rf build
!mkdir build
!tar xvf model.tar -C build
!rm -rf model.tar
%cd build

./
./codegen/
./codegen/host/
./codegen/host/include/
./codegen/host/include/tvmgen_default.h
./codegen/host/src/
./codegen/host/src/default_lib0.c
./codegen/host/src/default_lib1.c
./metadata.json
./parameters/
./parameters/default.params
./src/
./src/default.relay
./runtime/
./runtime/CMakeLists.txt
./runtime/include/
./runtime/include/checksum.h
./runtime/include/dlpack/
./runtime/include/dlpack/dlpack.h
./runtime/include/dmlc/
./runtime/include/dmlc/any.h
./runtime/include/dmlc/array_view.h
./runtime/include/dmlc/base.h
./runtime/include/dmlc/blockingconcurrentqueue.h
./runtime/include/dmlc/build_config_default.h
./runtime/include/dmlc/common.h
./runtime/include/dmlc/concurrency.h
./runtime/include/dmlc/concurrentqueue.h
./runtime/include/dmlc/config.h
./runtime/include/dmlc/data.h
./runtime/include/dmlc/endian.h
./runtime/include/dmlc/filesystem.h
./runtime/include/dmlc/input_split_shuffle.h
./runtime/include/dmlc/io.h
./runtime/include/dmlc/json.h
./runtime/include/dmlc/logging.h

### 4.2 實作基本的記憶體配置
建立 `crt_config.h` 與 `platform.c`。從 templates/crt_config.h.template 和 templates/platform.c.template 複製修改而得。

In [7]:
import shutil
import os

# 檔案來源與目標目錄
template_dir = "templates"
target_dir = "."

# 檔案名稱
files_to_copy = {
    "crt_config.h.template": "crt_config.h",
    "platform.c.template": "platform.c"
}

# 確保模板目錄存在
if not os.path.exists(template_dir):
    raise FileNotFoundError(f"模板目錄 '{template_dir}' 不存在，請確認目錄結構。")

# 複製檔案
for template_file, target_file in files_to_copy.items():
    source_path = os.path.join(template_dir, template_file)
    target_path = os.path.join(target_dir, target_file)

    if not os.path.exists(source_path):
        raise FileNotFoundError(f"模板檔案 '{source_path}' 不存在，請確認。")

    shutil.copy(source_path, target_path)

# 檢查目標檔案是否已成功建立
created_files = [file for file in files_to_copy.values() if os.path.exists(file)]
created_files

['crt_config.h', 'platform.c']

TVMLogf 和 TVMSystemLibEntryPoint 是 TVM CRT runtime 中預期的函式實作，通常需要在 `platform.c` 提供對應的實作。

這段程式碼的功能是自動檢查 `platform.c` 是否存在，然後在檔案中新增所需的定義（引入標頭檔案與設定工作區大小）和兩個函式（用於日誌輸出與模型執行的進入點），確保內容只新增一次並正確插入，最後保存修改結果。

- 在 `platform.c` 引入定義的變數並宣告 TVM_WORKSPACE_SIZE_BYTES 的大小。
- 在 platform.c 中定義 TVMSystemLibEntryPoint 和 TVMLogf

**實作 TVMLogf 函式**

TVM CRT 預設需要一個 TVMLogf 函式來輸出日誌（log）。你需要在自己的 platform.c 中自行實作此函式。

```c
void TVMLogf(const char* msg, ...) {
    va_list args;
    va_start(args, msg);
    vfprintf(stderr, msg, args);
    va_end(args);
}
```

這是一個簡單的實作，只是把 log 輸出到 stderr。

**實作 TVMSystemLibEntryPoint 函式**

這個符號通常在使用 system-lib 的情境下會被需要。
如果你的程式中需要 TVMSystemLibEntryPoint，你可以在 platform.c 或另一個 c 檔案中給它一個空實作，避免 undefined reference。通常它是用來載入 system-lib 的註冊入口。

```c
extern void* tvmgen_default___tvm_main__;

void* TVMSystemLibEntryPoint(void) {
  return &tvmgen_default___tvm_main__;
}
```

In [8]:
# 定義目標檔案名稱
platform_file = "platform.c"

# 檢查檔案是否存在
import os

if not os.path.exists(platform_file):
    raise FileNotFoundError(f"檔案 '{platform_file}' 不存在，請確認檔案名稱和目錄。")

# 新增的定義
new_define = "#include \"tvmgen_default.h\"\n#define TVM_WORKSPACE_SIZE_BYTES TVMGEN_DEFAULT_WORKSPACE_SIZE\n"

# 讀取原始檔案內容
with open(platform_file, "r") as file:
    lines = file.readlines()

# 確保定義只新增一次
if new_define not in lines:
    # 將定義新增到檔案開頭（通常是放在 include 後面）
    for i, line in enumerate(lines):
        if line.startswith("#include"):
            insertion_index = i + 1
            break
    else:
        insertion_index = 0  # 如果沒有 include，則放在檔案最開頭

    lines.insert(insertion_index, new_define)

# 定義要新增的函式內容
additional_functions = """
void TVMLogf(const char* msg, ...) {
    va_list args;
    va_start(args, msg);
    vfprintf(stderr, msg, args);
    va_end(args);
}

extern void* tvmgen_default___tvm_main__;

void* TVMSystemLibEntryPoint(void) {
  return &tvmgen_default___tvm_main__;
}
"""

# 確保函式只新增一次
if additional_functions.strip() not in "".join(lines):
    # 將函式新增到檔案結尾
    lines.append("\n")  # 確保前面有換行
    lines.append(additional_functions)

# 寫回檔案
with open(platform_file, "w") as file:
    file.writelines(lines)

# 確認檔案已更新
with open(platform_file, "r") as file:
    updated_content = file.readlines()

updated_content[-10:]  # 顯示最後幾行，確認修改結果


['    va_start(args, msg);\n',
 '    vfprintf(stderr, msg, args);\n',
 '    va_end(args);\n',
 '}\n',
 '\n',
 'extern void* tvmgen_default___tvm_main__;\n',
 '\n',
 'void* TVMSystemLibEntryPoint(void) {\n',
 '  return &tvmgen_default___tvm_main__;\n',
 '}\n']

## 5. C 推論實作
將這些檔案組裝成一個可在C原生環境下直接執行推論的程式。

### 5.1 準備必要檔案
- `tvmgen_default.h`：已經在 codegen/host/include/ 下。
- 模型程式碼：`default_lib0.c`、`default_lib1.c` (在 codegen/host/src/ 下)
- TVM CRT Runtime 原始碼：在 runtime/src/ 有多個子目錄。
- CRT 必要標頭：在 runtime/include/ 下
- `crt_config.h` 與 `platform.c`：可由 templates/crt_config.h.template 和 templates/platform.c.template 複製修改而得。你需要提供一個 `crt_config.h` 設定檔案以及一個最小化的 `platform.c` 去實作基本的記憶體配置。

### 5.2 撰寫 main.c（使用AOT介面進行推論）
當前擁有的資料夾結構是 microTVM 的 MLF 輸出，其內含完整的 model C code、header檔、以及 TVM CRT 的程式碼。透過：

1. 準備 main.c，包含 tvmgen_default.h 並呼叫模型執行函式
2. 編譯所有必要的 C 檔案及 runtime 原始碼
3. 提供 crt_config.h 與 platform.c（可從 templates 修改）

就能成功產生一個可獨立運行的 C 執行檔，直接進行推論而不依賴外部的 libtvm_runtime.so。

目前已經有了 `tvmgen_default.h`，可以撰寫一個 `main.c` 來呼叫 AOT 產生的函式介面。範例（以鳶尾花模型為例，假設輸入 shape=(1,4)，輸出 shape=(1,3)）：

In [ ]:
#include <stdio.h>
#include <tvmgen_default.h>

// 定義工作空間
uint8_t workspace[TVMGEN_DEFAULT_WORKSPACE_SIZE];

int main() {
    // 準備輸入及輸出資料
    float input_data[4] = {6.3, 3.3, 6.0, 2.5};
    int32_t output0_data;
    float output1_data[3];

    struct tvmgen_default_inputs inputs = {
        .float_input = (void*)input_data
    };

    struct tvmgen_default_outputs outputs = {
        .output0 = (void*)&output0_data,
        .output1 = (void*)output1_data
    };

    // 呼叫 run 函式，將 inputs & outputs 當作參數傳入
    int32_t result = tvmgen_default_run(&inputs, &outputs);
    if (result != 0) {
        fprintf(stderr, "Error: tvmgen_default_run failed with code %d\n", result);
        return -1;
    }

    printf("First Inference:\n");
    printf("Output0: %d\n", output0_data);
    printf("Output1: %f %f %f\n", output1_data[0], output1_data[1], output1_data[2]);

    return 0;
}

### 5.3 編譯整合
你需要將下列檔案一起編譯並連結：

- `main.c` (你自己寫的)
- `codegen/host/src/default_lib0.c`、`default_lib1.c` (模型程式碼)
- CRT runtime 的 C 檔案 (runtime/src/ 底下的檔案，如 `aot_executor.c`, `crt_runtime_api.c`,` graph_executor.c`, 等，需要依據你使用 AOT Executor 的設定挑選。若不確定，就將 runtime/src/runtime/crt/ 下的相關檔案全部編進去)
- 你的 `platform.c`（由 templates/platform.c.template 修改而來）
- 使用 `crt_config.h`（由 templates/crt_config.h.template 修改而來），你可以將其放在當前目錄並用 -I. 讓編譯器找到
- 提供 include 路徑給 -Icodegen/host/include、-Iruntime/include

**故障排除**

在 Windows 編譯過程中使用了 `__declspec(dllimport)` 導致符號預期從動態連結庫（DLL）匯入，而非在本地直接定義。

在 Windows 上，如果你預期建立單一可執行檔，而不是 DLL，建議在編譯時關閉 dllimport 修飾符。

你可以嘗試在編譯指令中加入定義，讓 TVM CRT 認為你在產生靜態連結的執行檔
```sh
-DTVM_DLL= -DTVM_CRT_STATIC_LIBRARY=1
```

**使用gcc編譯**

In [12]:
!gcc -O2 \
    -I./codegen/host/include \
    -I./runtime/include \
    -I. \
    main.c codegen/host/src/default_lib0.c codegen/host/src/default_lib1.c \
    runtime/src/runtime/crt/common/*.c \
    runtime/src/runtime/crt/aot_executor/*.c \
    runtime/src/runtime/crt/memory/*.c \
    platform.c \
    -lm \
    -o main

In [13]:
!./main

First Inference:
Output0: 2
Output1: 0.000000 0.000000 1.000000


**使用g++編譯(量測時間版本)**

如果想用 g++ 最終連結可以分兩步。首先使用 gcc 編譯 C 程式碼，g++ 僅用於連結：

- 用 gcc 編譯所有 C 檔產生 .o
- 再用 g++ 將這些 .o 與 main.cpp 連結在一起。

In [9]:
!gcc -O2 \
    -I./codegen/host/include \
    -I./runtime/include \
    -I. \
    -c codegen/host/src/default_lib0.c codegen/host/src/default_lib1.c \
    runtime/src/runtime/crt/common/*.c \
    runtime/src/runtime/crt/aot_executor/*.c \
    runtime/src/runtime/crt/memory/*.c \
    platform.c

In [10]:
!g++ -O2 *.o -lm main.cpp -I./codegen/host/include -o main -pthread
!rm -rf *.o

In [11]:
!./main

Average inference time for 3000 runs: 0.000103905 ms
Output0 (標籤): 2
Output1: 0.000035 0.009924 0.990042 


## Reference

- [後續可以嘗試用TVMC指令產生MLF](https://tvm.apache.org/docs/how_to/work_with_microtvm/micro_tvmc.html#compiling-a-tflite-model-to-a-model-library-format)